# Cell Health Data Segmentor
### Find center coordinates for cells from Cell Health data

#### Import libraries

In [1]:
#from cellpose.io import logger_setup
from cellpose import models, core, io, utils

import pathlib
import pandas as pd

### Set Up CellPose

In [2]:
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)
#logger_setup();

>>> GPU activated? 1


### Helper functions for segmenting Cell Health data

In [3]:
def get_nuclei_locations(DNA_image_path: pathlib.Path, save_path: pathlib.Path, cellpose_model: models.Cellpose) -> pd.DataFrame:
    """finds center X,Y of nuclei and saves as tsv file

    Args:
        DNA_image_path (pathlib.Path): path to DNA channel image
        save_path (pathlib.Path): path to save nuclei locations tsv
        cellpose_model (models.Cellpose): cellpose model for segmenting nuclei

    Returns:
        pd.DataFrame: dataframe with nuclei center coords
    """
    nuclei_data = []
    
    #use cellpose to get nuclei outlines
    frame_image = io.imread(DNA_image_path)
    masks, flows, styles, diams = cellpose_model.eval(frame_image, diameter=80, channels=[0,0], flow_threshold=0)
    outlines = utils.outlines_list(masks)
    
    for outline in outlines:
        centroid = outline.mean(axis=0)
        nucleus_data = {"Location_Center_X": centroid[0], "Location_Center_Y": centroid[1]}
        nuclei_data.append(nucleus_data)
        
    nuclei_data = pd.DataFrame(nuclei_data)
    return nuclei_data

def segment_cell_health(data_path: pathlib.Path, save_path: pathlib.Path, cellpose_model_DNA: models.Cellpose, cellpose_model_actin: models.Cellpose):
    """segments cell health data from data_path and save segmentation data in save_path using cellpose_model

    Args:
        data_path (pathlib.Path): load path for cell health data
        save_path (pathlib.Path): save path for segmentation data
        cellpose_model_DNA (models.Cellpose): cell pose model to use for segmenting DNA
        cellpose_model_actin (models.Cellpose): cell pose model to use for segmentating actin
    """
    
    for plate_path in data_path.iterdir():
        print(f"Segmenting plate {plate_path.name}")
        for image_folder in plate_path.iterdir():
            for image_file in image_folder.iterdir():
                if ".tiff" in image_file.name:
                    
                    if "-ch1" in image_file.name:
                        nuc_save_path = str(image_file).replace("cell-health", "cell-health-segmented")
                        nuc_save_path = nuc_save_path.replace(".tiff", "-segmented.tsv")
                        nuc_save_path = pathlib.Path(nuc_save_path)
                        
                        if not nuc_save_path.is_file():
                            print(f"Segmenting {nuc_save_path.name}")
                            nuc_save_path.parents[0].mkdir(parents=True, exist_ok=True)
                            nuc_locations = get_nuclei_locations(image_file, nuc_save_path, cellpose_model_DNA)
                            nuc_locations.to_csv(nuc_save_path, sep="\t")
                        else:
                            print(f"{nuc_save_path.name} already exists!")

In [4]:
data_path = pathlib.Path("/media/roshankern/63af2010-c376-459e-a56e-576b170133b6/data/cell-health/")
save_path = pathlib.Path("/media/roshankern/63af2010-c376-459e-a56e-576b170133b6/data/cell-health-segmented/")
cellpose_model_DNA = models.Cellpose(gpu=True, model_type='cyto')
cellpose_model_actin = models.Cellpose(gpu=True, model_type='cyto')

segment_cell_health(data_path, save_path, cellpose_model_DNA, cellpose_model_actin)

Segmenting plate SQ00014617
r07c18f04p01-ch1sk1fk1fl1-segmented.tsv already exists!
r03c16f02p01-ch1sk1fk1fl1-segmented.tsv already exists!
r03c02f04p01-ch1sk1fk1fl1-segmented.tsv already exists!
r05c04f08p01-ch1sk1fk1fl1-segmented.tsv already exists!
r06c19f07p01-ch1sk1fk1fl1-segmented.tsv already exists!
r16c14f08p01-ch1sk1fk1fl1-segmented.tsv already exists!
r04c24f06p01-ch1sk1fk1fl1-segmented.tsv already exists!
r06c19f05p01-ch1sk1fk1fl1-segmented.tsv already exists!
r08c10f04p01-ch1sk1fk1fl1-segmented.tsv already exists!
r07c03f05p01-ch1sk1fk1fl1-segmented.tsv already exists!
r01c15f07p01-ch1sk1fk1fl1-segmented.tsv already exists!
r07c18f01p01-ch1sk1fk1fl1-segmented.tsv already exists!
r08c18f05p01-ch1sk1fk1fl1-segmented.tsv already exists!
r05c21f07p01-ch1sk1fk1fl1-segmented.tsv already exists!
r07c04f06p01-ch1sk1fk1fl1-segmented.tsv already exists!
r11c21f02p01-ch1sk1fk1fl1-segmented.tsv already exists!
r04c14f04p01-ch1sk1fk1fl1-segmented.tsv already exists!
r06c05f02p01-ch1sk1f

/tmp/ipykernel_296115/1036601208.py:20: RuntimeWarning: Mean of empty slice.
  centroid = outline.mean(axis=0)
/home/roshankern/anaconda3/envs/1.segment-data-cell-health/lib/python3.8/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


Segmenting r06c23f09p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r15c03f04p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r03c06f09p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r05c15f02p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r05c08f02p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r08c11f08p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r05c01f01p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r16c03f02p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r13c21f01p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r04c15f01p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r12c11f09p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r05c24f04p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r16c20f02p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r03c09f05p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r12c01f05p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r09c02f06p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r08c12f06p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r02c12f02p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r14c03f01p01-ch1sk1fk1fl1-segmented.tsv
Segmenting r10c15f09p01-ch1sk1f